It turns out the kernel I wrote in "naive_matmul_kernel.ipynb" isn't correct. It's not technically incorrect, <br>
but it's incorrect in terms of what I intended to do. I wanted to do row-major matmul, but I ended up doing column-major matmul. <br><br>
https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Row_and_column_major_order.svg/340px-Row_and_column_major_order.svg.png<br><br>
So, I corrected it without expecting anything, but it ended up giving me great performance improvement. This notebook essentially <br>
explores the "why" behind the performance improvement. <br>

The "why" is explored at the end of the notebook.

In [1]:
import pyopencl as cl
import numpy as np
import time 

In [2]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

num_of_test_loops = 10

Col major kernel.

In [3]:
krnl_col_major = """
__kernel void mat_mul_col(__global float *res, __global float *first, __global float *second){
    int idx_row = get_global_id(0);
    int idx_col = get_global_id(1);
    int size = get_global_size(0);
    float val = 0; 
    for(int i=0; i<size; ++i){
       //btw, memory is contiguously allocated, so indexing like this. Stride = (size, 1).
        val += first[size*idx_row+i] * second[size*i+idx_col];
    }
    res[size*idx_row+idx_col] = val;
}
"""
krnl_prog_col = cl.Program(ctx, krnl_col_major).build()

Row major kernel.

In [4]:
krnl_row_major = """
__kernel void mat_mul_row(__global float *res, __global float *first, __global float *second){
    // just flip 1 to 0 and 0 to 1 from above. 
    // This is what was intended. 
    int idx_row = get_global_id(1); 
    int idx_col = get_global_id(0);
    
    int size = get_global_size(0); // we're working with nxn matrix, so doesn't matter
                                   // whether we do (0) or (1) in get_global_size. 
    float val = 0; 
    for(int i=0; i<size; ++i){
       //btw, memory is contiguously allocated, so indexing like this. Stride = (size, 1).
        val += first[size*idx_row+i] * second[size*i+idx_col];
    }
    res[size*idx_row+idx_col] = val;
}
"""
krnl_prog_row = cl.Program(ctx, krnl_row_major).build()

In [5]:
def memory_stuff():
    mf = cl.mem_flags

    sz = 4096 # the size of the matrix is sz x sz. you can play with this. 

    first_host = np.random.uniform(0, 1, size = (sz, sz)).astype(np.float32)
    first_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=first_host)

    second_host = np.random.uniform(0, 1, size = (sz, sz)).astype(np.float32)
    second_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=second_host)

    result = cl.Buffer(ctx, mf.WRITE_ONLY, first_host.nbytes)
    
    return first_device, second_device, result, sz

Running col major kernel. 16x16 workgroup size is better for col major matmul for some reason. If I do 8x8, it's horrendously slow.

In [6]:
krnl_col_obj = krnl_prog_col.mat_mul_col

avg_time_col = 0

for i in range(num_of_test_loops):
    first_device, second_device, result, sz = memory_stuff()
    
    start_time = time.perf_counter()
    
    event = krnl_col_obj(queue, (sz, sz), (16, 16), result, first_device, second_device)
    event.wait()
    
    end_time = time.perf_counter()
    
    avg_time_col += end_time - start_time

Running row major kernel.

In [7]:
krnl_row_obj = krnl_prog_row.mat_mul_row

avg_time_row = 0

for i in range(num_of_test_loops):
    first_device, second_device, result, sz = memory_stuff()
    
    start_time = time.perf_counter()
    
    event = krnl_row_obj(queue, (sz, sz), (8, 8), result, first_device, second_device)
    event.wait()
    
    end_time = time.perf_counter()
    
    avg_time_row += end_time - start_time

Comparing speeds.

In [8]:
avg_time_col /= num_of_test_loops 
avg_time_row /= num_of_test_loops 

speedup = avg_time_col/avg_time_row

print(f"Average speed for col major = {avg_time_col} sec.")
print(f"Average speed for row major = {avg_time_row} sec.")
print(f"Row major matmul is {speedup}x as fast as col major matul.")

Average speed for col major = 5.562838355402346 sec.
Average speed for row major = 0.8338460477942136 sec.
Row major matmul is 6.6713014592055835x as fast as col major matul.


As you can see, row major is significantly faster than col major. The speed of row major is on par with numpy's CPU implementation -- even slightly better actually. Why is row major faster than col major? It is probably something to do with caching. <br>

<strong>Assumption</strong>: Workgroups are mapped onto the GPU in row-major order. <br>

<strong>Probability calculations</strong>: <br>

<i>For "result" array</i>:<br>

Prob. of a cell from the same row being accessed in row major matmul (Pr_R) = 1024/(1024&times;64) = 1/64 <br>
Prob. of a cell from the same row being accessed in col major matmul (Pc_R) = 64/(1024&times;64) = 1/1024 <br>

"Pr_R" is 16 times more likely than "Pc_R". <br>

As elements are stored contiguously row-major, "Pr_R" being relatively higher than "Pc_R" implies higher cache hits -- chew on this. Think of locality of reference. <br>

<i>For "first" array</i>:<br>

Prob. of a cell from the same row being accessed in row major matmul (Pr_F) = (1024&times;4096)/(1024&times;4096&times;64) = 1/64 <br>
Prob. of a cell from the same row being accessed in col major matmul (Pc_F) = (64&times;4096)/(64&times;4096&times;1024) = 1/1024 <br>

"Pr_F" is 16 times more likely than "Pc_F". <br>

By the same reasoning as above, here too there are higher cache hits. 

<i>For "second" array</i>:<br>

Prob. of a cell from the same row being accessed in row major matmul (Pr_S) = (1024&times;64)/(1024&times;64&times;4096) = 1/4096 <br>
Prob. of a cell from the same row being accessed in col major matmul (Pc_S) = (64&times;1024)/(64&times;1024&times;4096) = 1/4096 <br>

Here, "Pr_S" and "Pc_S" are equal. So, the cache hits aren't going to be different.

<strong>Overall Conclusion</strong>: Even though cache hits aren't going to be different for the "second" array, the fact that "Pr_R" is 16 times more likely than "Pc_R", and "Pr_F" is 16 times more likely than "Pc_F", the increased cache hits here are probably going to be significant. This, I think, explains the speedup. 